In [ ]:
import os
import chess
import chess.pgn
import pandas as pd

In [ ]:
input_file = os.path.abspath('../data/raw/lichess_db_standard_rated_2021-03.pgn')
output_file = os.path.abspath('../data/indexed/positions.parquet')

num_games = 100
num_positions_per_game = 10

In [ ]:
positions_columns = [
        'fen',
        'white_wins',
        'black_wins',
        'draws',
    ]
positions = pd.DataFrame(columns=positions_columns)
positions.set_index('fen', inplace=True)

moves = pd.DataFrame(
    columns=[
        'fen',
        'move',
        'times_played'
    ],
)
moves.set_index(['fen', 'move'], inplace=True)

In [ ]:
# Parse and add games
with open(input_file) as pgn_file:
	board = chess.Board()

	for _ in range(num_games):
		board.reset()
		game = chess.pgn.read_game(pgn_file)
		result = game.headers['Result']
		
		for move in game.mainline_moves():
			fen = board.fen()
		
			if fen not in positions.index:
				positions.loc[fen] = 0
			
			if result == "1-0":
				positions.at[fen, "white_wins"] += 1
			elif result == "1/2-1/2":
				positions.at[fen, "draws"] += 1
			elif result == "0-1":
				positions.at[fen, "black_wins"] += 1
			else:
				raise f"Invalid result: {result}"
			
			board.push(move)


In [ ]:
positions.to_parquet(output_file)